In [ ]:
# 导入必要的工具
from jax import numpy as jnp
from jax import random
from jax import grad, jit, vmap
from jax import jacfwd, jacrev
from jax import jacobian
from jax import hessian
import numpy as np

# 下面用一个简单的例子来说明 Jax 相对于 numpy 的性能优势

In [ ]:
# 用 Jax 写 selu 函数 并且在随机数上进行测试
def jselu(x, alpha=1.67, lmbda=1.05):
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = jnp.arange(5.0)
print(jselu(x))

key = random.PRNGKey(1701)
x = random.normal(key, (1_000_000,))
%timeit jselu(x).block_until_ready()

In [ ]:
# 用 numpy 写 selu 函数 并且在随机数上进行测试
def selu(x, alpha=1.67, lmbda=1.05):
    return lmbda * np.where(x > 0, x, alpha * np.exp(x) - alpha)

x = np.arange(5.0)
print(selu(x))

x = np.random.normal(size=(1_000_000,))
%timeit selu(x)

In [ ]:
# 用 jit 编译 jselu 函数, 性能更好了
selu_jit = jit(jselu)
_ = selu_jit(x)  # compiles on first call
%timeit selu_jit(x).block_until_ready()

# 下面是随机数和种子的学习

In [ ]:
# 测试学习 numpy 中的全局和局部随机数生成器
np.random.seed(2026)
y = np.random.normal(size=2)
print(y)

rd = np.random.default_rng(3)
x = rd.normal(size=2)
print(x)

z = np.random.uniform(size=3)
print(z)

In [ ]:
# 测试 Jax 中的随机数生成器并且学着用 key 和分裂操作

# 创建主随机数密钥
key = random.PRNGKey(1)
print(key)

# 分裂主密钥生成两个子密钥
key, subkey = random.split(key)

# 使用子密钥生成随机数
x = random.normal(subkey, shape=(4,))

key, subkey = random.split(key)
y = random.uniform(subkey, shape=(4,))

print("正态分布随机数:", x)
print("均匀分布随机数:", y)

# 下面是 Jax 中的自动微分

In [ ]:
# 自动微分的一个初步例子
# grad 函数是梯度，可以有多维输入但是只能有一维输出
def sum_logistic(x):
  return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))

x_small = jnp.arange(3.)
derivative_fn = grad(sum_logistic)
print(x_small, derivative_fn(x_small))

In [ ]:
# grad 和 git 可以随机嵌套使用
# 这个单元格比较了是否用 jit 的不同性能

test_fun = grad(jit(grad(jit(grad(sum_logistic)))))
_ = test_fun(1.0)  # compiles on first call

%timeit test_fun(1.0).block_until_ready()
%timeit grad(grad(grad(sum_logistic)))(1.0).block_until_ready()

In [ ]:
# 因为 grad 处理的函数只能有一维输出，所以我们只能使用 0.0 作为测试例子
grad_exp = grad(jnp.exp)
print(grad_exp(0.0))
# print(grad_exp(0.0, 1.0))
# 这个例子是错误的
# 因为当喂进去一个二维向量，那么 jnp.exp 的输出也会使用一个二维向量，所以再嵌套grad就会报错


# 下面是一个正确的例子
# 作为替代，可以使用 vmap 来处理多维输入
grad_exp = vmap(grad(jnp.exp))
print(grad_exp(x_small))

# 或者使用 jacobian 来处理多维输入
# jacobian 处理的函数可以有多维输入和多维输出

print(jacobian(jnp.exp)(x_small))
# 或者继续取对角线得到和向量化一样的结果
print(jnp.diag(jacobian(jnp.exp)(x_small)))

In [ ]:
# 尝试两种不同形式的 hessian
def hessian1(fun):
  return jit(jacfwd(jacrev(fun)))
def hessian2(fun):
  return jit(jacrev(jacfwd(fun)))

_ = hessian1(jnp.exp)(x_small)  # compiles on first call
_ = hessian2(jnp.exp)(x_small)  # compiles on first call

In [ ]:
# 比较两种不同形式的自实现 hessian 以及 jax 中的 hessian
%timeit hessian1(sum_logistic)(x_small).block_until_ready()
%timeit hessian2(sum_logistic)(x_small).block_until_ready()

# 下面是 jax 中的 hessian
%timeit hessian(sum_logistic)(x_small).block_until_ready()

# 可以看到 jax 提供的 hessian 的性能是最好的

# 下面是 Jax 中的自动向量化

In [ ]:
key = random.PRNGKey(2025)
key, subkey1, subkey2 = random.split(key, 3)
mat = random.normal(subkey1, (150, 100))
batched_x = random.normal(subkey2, (10, 100))

def apply_matrix(x):
  return jnp.dot(mat, x)

In [ ]:
def naively_batched_apply_matrix(v_batched):
  return jnp.stack([apply_matrix(v) for v in v_batched])

print(apply_matrix(batched_x[0]).shape)

print('Naively batched')
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

In [ ]:
@jit
def vmap_batched_apply_matrix(batched_x):
  return vmap(apply_matrix)(batched_x)

_ = vmap_batched_apply_matrix(batched_x)  # compiles on first call

np.testing.assert_allclose(naively_batched_apply_matrix(batched_x),
                           vmap_batched_apply_matrix(batched_x), atol=1E-4, rtol=1E-4)
print('Auto-vectorized with vmap')
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()